In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Database configuration
db_config = {
    'user': 'praktikum_student',
    'pwd': 'Sdf4$2;d-d30pp',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,
    'db': 'data-science-final'
}

# Create the database connection
connection_string = (
    f"postgresql://{db_config['user']}:{db_config['pwd']}@"
    f"{db_config['host']}:{db_config['port']}/{db_config['db']}"
)
engine = create_engine(connection_string)

query = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';"
available_tables = pd.read_sql_query(query, con=engine)
display(available_tables)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,steel,data_arc,praktikum_admin,None,False,False,False,False
1,steel,data_bulk,praktikum_admin,None,False,False,False,False
2,steel,data_bulk_time,praktikum_admin,None,False,False,False,False
3,steel,data_gas,praktikum_admin,None,False,False,False,False
4,steel,data_temp,praktikum_admin,None,False,False,False,False
5,steel,data_wire,praktikum_admin,None,False,False,False,False
6,steel,data_wire_time,praktikum_admin,None,False,False,False,False
7,telecom,contract,praktikum_admin,None,False,False,False,False
8,telecom,internet,praktikum_admin,None,False,False,False,False
9,telecom,personal,praktikum_admin,None,False,False,False,False


In [3]:
# Function to load data from a table
def load_data(schema, table_name):
    query = f"SELECT * FROM {schema}.{table_name}"
    return pd.read_sql_query(query, con=engine)

# Load data from each table in the 'telecom' schema
contract_data = load_data('telecom', 'contract')
personal_data = load_data('telecom', 'personal')
internet_data = load_data('telecom', 'internet')
phone_data = load_data('telecom', 'phone')

# Display the first few rows of one of the tables as an example
display(contract_data.head())
display(personal_data.head())
display(internet_data.head())
display(phone_data.head())

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,4472-LVYGI,2020-02-01,None,Two year,Yes,Bank transfer (automatic),52.55,NaN
1,3115-CZMZD,2020-02-01,None,Two year,No,Mailed check,20.25,NaN
2,3668-QPYBK,2018-08-09,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15
3,5709-LVOEQ,2020-02-01,None,Two year,No,Mailed check,80.85,NaN
4,9237-HQITU,2019-01-26,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


,c1,c2,c3,c4,c5,c6,c7,c8
0,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
1,7590-VHVEG,DSL,No,Yes,No,No,No,No
2,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
3,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
4,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No


,c1,c2
0,customerID,MultipleLines
1,5575-GNVDE,No
2,3668-QPYBK,No
3,9237-HQITU,No
4,9305-CDSKC,Yes
